## Лекция 3  NER

### __Задача 1__:

Реализуйте 2 функции препроцессинга:

- Удалить именованные сущности с помощью natasha (https://github.com/natasha/yargy)
- Удалить именованные сущности с помощью deepmipt (https://github.com/deepmipt/ner)

In [33]:
test_text = '''Добрый день!

Пожалуйста, уточните, на данный момент для посещения Камчатки требуется ли предоставление отрицательных тестов на COVID19 или какие-либо другие документов, помимо паспорта гражданина РФ для прибывающих из Москвы регулярным рейсом?

Нужно ли оформлять какие-либо пропуски для пребывания на тер-рии Камчатки? Целью поездки явлеятся туризм, проживание заранее оплачивается, экскурсионные услуги будут организованы непосредственно через отель. Есть ли другие ограничения или меры, о которых необходимо знать туристам, которые отправляются на Камчатку (11-16 августа)


Заранее спасибо

Елена Мягкова
'''

In [34]:
from natasha import (
    Segmenter,
    
    NewsEmbedding,
    NewsNERTagger,

    Doc
)

segmenter = Segmenter()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

In [35]:
import re
from razdel import tokenize
from nltk.corpus import stopwords
stops = stopwords.words("russian")
stops.extend(['здравствуйте', 'добрый', 'день'])

from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

def tokenizer(text_data):
    tokens = [_.text for _ in list(tokenize(str(text_data).lower())) if not re.search('[^а-яА-ЯёЁa-zA-z]', _.text)]
    return " ".join(tokens)

In [36]:
from tqdm.auto import tqdm
tqdm.pandas()

In [64]:
def preprocess_with_natasha(text: str) -> str:
    #text = text.replace('\n', ' ')
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    for span in doc.spans:
        text = text.replace(span.text, "")
    new_text = re.sub("\s+", " ", text)
    
    return re.sub(r"([^\w\s])", r" \1 ", new_text)

In [65]:
preprocess_with_natasha(test_text)

'Добрый день !  Пожалуйста ,  уточните ,  на данный момент для посещения требуется ли предоставление отрицательных тестов на COVID19 или какие - либо другие документов ,  помимо паспорта гражданина для прибывающих из регулярным рейсом ?  Нужно ли оформлять какие - либо пропуски для пребывания на тер - рии  ?  Целью поездки явлеятся туризм ,  проживание заранее оплачивается ,  экскурсионные услуги будут организованы непосредственно через отель .  Есть ли другие ограничения или меры ,  о которых необходимо знать туристам ,  которые отправляются на  ( 11 - 16 августа )  Заранее спасибо '

In [41]:
from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_rus_bert, download=True)

2020-09-28 21:34:20.70 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/bert/rubert_cased_L-12_H-768_A-12_v1.tar.gz download because of matching hashes
2020-09-28 21:34:22.247 INFO in 'deeppavlov.download'['download'] at line 117: Skipped http://files.deeppavlov.ai/deeppavlov_data/ner_rus_bert_v1.tar.gz download because of matching hashes
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data] 

In [66]:
def preprocess_with_deepmipt(text: str) -> str:
    new_text = []
    res = ner_model([text])
    for token, entity in zip(res[0][0], res[1][0]):
        if entity == "O":
            new_text.append(token)
    return re.sub(r"\s+", " ", ' '.join(new_text))

In [67]:
preprocess_with_deepmipt(test_text)

'Добрый день ! Пожалуйста , уточните , на данный момент для посещения требуется ли предоставление отрицательных тестов на COVID19 или какие - либо другие документов , помимо паспорта гражданина для прибывающих из регулярным рейсом ? Нужно ли оформлять какие - либо пропуски для пребывания на тер - рии ? Целью поездки явлеятся туризм , проживание заранее оплачивается , экскурсионные услуги будут организованы непосредственно через отель . Есть ли другие ограничения или меры , о которых необходимо знать туристам , которые отправляются на ( 11 - 16 августа ) Заранее спасибо '

### __Задача 2__:    
На предыдущем занятии вы реализовывали функции поиска ближайших ответов на запросы через TF-IDF и BM25. 
Сравните качество нахождения верного ответа для обоих методов в трех случаях:
- с функцией ```preprocess_with_natasha```
- с функцией ```preprocess_with_deepmipt```
- без препроцессинга

Для измерения качества используйте метрику accuracy. Считаем, что ответ верный, если он входит в топ-1.

In [73]:
import pandas as pd
import numpy as np

Готовлю данные

In [71]:
answers_data = pd.read_excel("answers_base.xlsx")

question_index = {}
for question_chunk, answer_n in answers_data[["Текст вопросов", "Номер связки"]].values:
    questions = question_chunk.split('\n')
    for q in questions:
        question_index[q] = answer_n
            
question_df = pd.DataFrame({"question": q, "index": i} for q,i in question_index.items())
question_df.head()

,index,question
0,57,У ребенка в школе продлили каникулы. Могу ли я...
1,57,Больничный лист?
2,57,"Есть ли компенсация, в случае если есть разниц..."
3,57,как оплачивается больничный при коронавирусе?
4,57,"Я контактный, дадут ли больничный?"


Собираю bm25 в одну функцию от названия колонки в датафрейме с данными

In [118]:
from sklearn.feature_extraction.text import CountVectorizer
from math import log

def new_tf(count_matrix):
    return (count_matrix * (k+1)) / (count_matrix + k * (1 - b + b * (length/avgdl) ))

def idf(i):
    res = log( (N - n[0, i] + 0.5) / (n[0, i] + 0.5) )
    return res

def bm25_search(q):
    q_vec = count_vectorizer.transform([q]).toarray()
    mask = q_vec * idf_vector
    res = np.dot(tf_matrix, mask.T)
    index = np.argmax(res)
    return answers_data["Номер связки"][index]

def bm25(data_col):

    count_vectorizer = CountVectorizer()
    corpora = answers_data["Текст ответа"]
    count_matrix = count_vectorizer.fit_transform(corpora)
    #count_matrix.shape

    k = 2.0
    b = 0.75

    length = count_matrix.sum(axis=1)
    avgdl = length.mean()


    tf_matrix = new_tf(count_matrix)
    #tf_matrix.shape

    n = (tf_matrix != 0).sum(axis=0)
    N = count_matrix.shape[0]

    idf_vector = np.array([idf(i) for i in range(count_matrix.shape[1])])
    #idf_vector.shape
    
    return question_df[data_col].progress_apply(bm25_search)

Собираю TF-IDF в одну функцию от названия колонки в датаврейме с данными

In [119]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data_col):
    vectorizer = TfidfVectorizer(analyzer=str.split)
    corpora = answers_data["Текст ответа"]
    X = vectorizer.fit_transform(corpora)
    #X.shape
    
    def get_answer(querry):
        q_vec = vectorizer.transform([querry]).toarray()
        res = np.dot(X.toarray(), q_vec.T)
        index = np.argmax(res)
        return answers_data["Номер связки"][index]
    
    return question_df[data_col].progress_apply(get_answer)

Делаю два препроцессинга

In [112]:
question_df["natasha"] = question_df["question"].progress_apply(preprocess_with_natasha)
question_df["deeppavlov"] = question_df["question"].progress_apply(preprocess_with_deepmipt)

Делаю таблицу для результатов подсчета accuracy

In [107]:
col_names = ["question", "natasha", "deeppavlov"]

res_df = pd.DataFrame(index=["tf-idf", "bm25"], columns=col_names)
res_df

,question,natasha,deeppavlov
tf-idf,NaN,NaN,NaN
bm25,NaN,NaN,NaN


Заполняю все ячейки, применяя два разных поиска к данным с термя разными препроцессингами

In [120]:
for col_name in col_names:
    question_df["pred_"+col_name+"_bm25"] = bm25(col_name)
    question_df["pred_"+col_name+"_tfidf"] = tfidf(col_name)
    
    res_df[col_name]["tf-idf"] = ((question_df["pred_"+col_name+"_tfidf"] == question_df['index']).sum() / len(question_df))
    res_df[col_name]["bm25"] = ((question_df["pred_"+col_name+"_bm25"] == question_df['index']).sum() / len(question_df))

Итоговые значения accuracy

In [130]:
res_df

,question,natasha,deeppavlov
tf-idf,0.238281,0.291667,0.291667
bm25,0.332031,0.329427,0.330729


### __Задача 3__:    
Улучшить правила в natasha. Написать правила, которые ловят даты в следующих примерах и пересчитать статистику из Задачи 2:
- Уехал 8-9 ноября в Сочи
- Уезжаю 5 числа                           
- 20го сентября заболел

Пример можно посмотреть тут: https://github.com/natasha/yargy